# Regression Week 2: Multiple Regression (Interpretation)
##### salimt

The goal of this first notebook is to explore multiple regression and feature engineering with existing Turi Create functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in Turi Create functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up Turi Create

In [1]:
import turicreate

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = turicreate.SFrame('home_data.sframe/')

# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let Turi Create pick a random seed for you).  

In [3]:
train_data,test_data = sales.random_split(.8,seed=0)

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [4]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = turicreate.linear_regression.create(train_data, target = 'price', features = example_features, 
                                                    validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 3

Number of unpacked features : 3

...

| Iteration | Passes   | Elapsed Time | Training Max Error | Training Root-Mean-Square Error |

+-----------+----------+--------------+--------------------+---------------------------------+

| 1         | 2        | 1.036021     | 4146407.600631     | 258679.804477                   |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [5]:
example_weight_summary = example_model.coefficients
print (example_weight_summary)

+-------------+-------+---------------------+--------------------+
|     name    | index |        value        |       stderr       |
+-------------+-------+---------------------+--------------------+
| (intercept) |  None |  87910.07249240018  | 7873.338143401675  |
| sqft_living |  None |  315.40344055209926 | 3.4557003258547327 |
|   bedrooms  |  None | -65080.215552828566 | 2717.456854420713  |
|  bathrooms  |  None |  6944.020192639225  | 3923.1149314414847 |
+-------------+-------+---------------------+--------------------+
[4 rows x 4 columns]



# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing Turi Create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [6]:
example_predictions = example_model.predict(train_data)
print (example_predictions[0]) # should be 271789.505878

271789.5058780308


# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [10]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    predictions = model.predict(data)
    # Then compute the residuals/errors then square and add them up
    RSS = sum((outcome-predictions)**2)
    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [15]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
#print (rss_example_train) 
print ('{:.2e}'.format(rss_example_train)) # should be 2.7376153833e+14

2.74e+14


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [16]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [28]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [47]:
def f(v, p1, p2): return p1*p2 if v=='m' else p1+p2

# create the remaining 3 features in both TEST and TRAIN data
train_data['bed_bath_rooms'] = map(lambda x,y:x*y,train_data['bedrooms'],train_data['bathrooms'])
test_data['bed_bath_rooms'] = map(lambda x,y:x*y,test_data['bedrooms'],test_data['bathrooms'])

train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))

train_data['lat_plus_long'] = map(lambda x,y:x+y,train_data['lat'],train_data['long'])
test_data['lat_plus_long'] = map(lambda x,y:x+y,test_data['lat'],test_data['long'])

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [48]:
print("%.2f" % (sum(test_data['lat_plus_long'])/len(test_data['lat_plus_long'])))

-74.65


In [32]:
print("%.2f" % (sum(test_data['bed_bath_rooms'])/len(test_data['bed_bath_rooms'])))

7.50


In [33]:
print("%.2f" % (sum(test_data['log_sqft_living'])/len(test_data['log_sqft_living'])))

7.55


In [34]:
print("%.2f" % (sum(test_data['bedrooms_squared'])/len(test_data['bedrooms_squared'])))

12.45


# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [35]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using turicreate.linear_regression.create() and look at the value of the weights/coefficients:

In [36]:
# Learn the three models: (don't forget to set validation_set = None)
model_1 = turicreate.linear_regression.create(train_data, target = 'price', features = model_1_features, 
                                                    validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 5

Number of unpacked features : 5

...

+-----------+----------+--------------+--------------------+---------------------------------+

| 1         | 2        | 0.027329     | 4074878.213085     | 236378.596454                   |

| 2         | 3        | 0.044301     | 4074878.213108     | 236378.596455                   |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

In [37]:
model_2 = turicreate.linear_regression.create(train_data, target = 'price', features = model_2_features, 
                                                    validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 6

Number of unpacked features : 6

...

+-----------+----------+--------------+--------------------+---------------------------------+

| 1         | 2        | 0.025623     | 4014170.932932     | 235190.935428                   |

| 2         | 3        | 0.067002     | 4014170.932936     | 235190.935429                   |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

In [38]:
model_3 = turicreate.linear_regression.create(train_data, target = 'price', features = model_3_features, 
                                                    validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 9

Number of unpacked features : 9

...

| Iteration | Passes   | Elapsed Time | Training Max Error | Training Root-Mean-Square Error |

+-----------+----------+--------------+--------------------+---------------------------------+

| 1         | 2        | 0.017302     | 3190659.608857     | 228020.710401                   |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

In [39]:
# Examine/extract each model's coefficients:
model1_weight_summary = model_1.coefficients
print (model1_weight_summary)

+-------------+-------+---------------------+--------------------+
|     name    | index |        value        |       stderr       |
+-------------+-------+---------------------+--------------------+
| (intercept) |  None |  -56140675.74114401 | 1649985.4204360154 |
| sqft_living |  None |  310.2633257769215  | 3.1888296040774544 |
|   bedrooms  |  None | -59577.116067596675 | 2487.279773224323  |
|  bathrooms  |  None |  13811.840541653153 | 3593.5421329700102 |
|     lat     |  None |  629865.7894714825  | 13120.710032334237 |
|     long    |  None | -214790.28516470865 | 13284.285161661053 |
+-------------+-------+---------------------+--------------------+
[6 rows x 4 columns]



In [40]:
model2_weight_summary = model_2.coefficients
print (model2_weight_summary)

+----------------+-------+---------------------+--------------------+
|      name      | index |        value        |       stderr       |
+----------------+-------+---------------------+--------------------+
|  (intercept)   |  None |  -54410676.11184223 | 1650405.1655748067 |
|  sqft_living   |  None |  304.4492980555793  | 3.202175356372319  |
|    bedrooms    |  None | -116366.04322944849 | 4805.549665487809  |
|   bathrooms    |  None |  -77972.33051298735 | 7565.059910961687  |
|      lat       |  None |   625433.834939811  | 13058.353097257446 |
|      long      |  None | -203958.60293747584 | 13268.12837210344  |
| bed_bath_rooms |  None |  26961.624908951988 | 1956.3656155607707 |
+----------------+-------+---------------------+--------------------+
[7 rows x 4 columns]



**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [41]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
model_1_train = get_residual_sum_of_squares(model_1, train_data, train_data['price'])
print (model_1_train)

971328233545429.9


In [42]:
model_2_train = get_residual_sum_of_squares(model_2, train_data, train_data['price'])
print (model_2_train)

961592067857504.1


In [43]:
model_3_train = get_residual_sum_of_squares(model_3, train_data, train_data['price'])
print (model_3_train)

903854036957249.4


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [44]:
# Compute the RSS on TESTING data for each of the three models and record the values:
model_1_test = get_residual_sum_of_squares(model_1, test_data, test_data['price'])
print (model_1_test)

226568089093161.16


In [45]:
model_2_test = get_residual_sum_of_squares(model_2, test_data, test_data['price'])
print (model_2_test)

224368799993971.03


In [46]:
model_3_test = get_residual_sum_of_squares(model_3, test_data, test_data['price'])
print (model_3_test)

255723991614121.44


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.